Amplitude Test
=========

The goal is to test the amplitude calculated on numpy arrays with the serial evaluation already implemented.

In [ ]:
from __future__ import print_function
import numpy as np
import sftrident as sf
from sftrident.amplitude2 import amplitude
import time
timesNP = {}
timesSQ = {}

In [ ]:

def buildArr_lcc(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-5):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return [ss,p1m,p1x,p1y,p2m,p2x,p2y]


def buildArr_lcc_single(eps=1e-5):
    ss = np.array([3.1])
    p1m = np.array([eps])
    p1x = np.array([-2.0])
    p1y = np.array([-2.0])
    p2m = np.array([eps])
    p2x = np.array([-2.0])
    p2y = np.array([-2.0])
    return [ss,p1m,p1x,p1y,p2m,p2x,p2y]


def buildArr_sph(Nss,Ne1,Nc1,Np1,Ne2,Nc2,Np2,eps=1e-5):
    ss = np.linspace(3.1,4.1,Nss)
    e1 = np.linspace(1.0+eps,5,Ne1)
    c1 = np.linspace(-1.0,1.0,Nc1)
    p1 = np.linspace(0.0,2.0*np.pi,Np1)
    e2 = np.linspace(1.0+eps,5,Ne2)
    c2 = np.linspace(-1.0,1.0,Nc2)
    p2 = np.linspace(0.0,2.0*np.pi,Np2)
    return [ss,e1,c1,p1,e2,c2,p2]

mass = 1.0
Narr1 = (2,3,2,5,6,7,8)
#Narr1 = (2,2,2,2,2,2,2)
frame = 'lab'
tempDPHI = 50.0

arrLCC = buildArr_lcc(*Narr1)
#arrSPH = buildArr_sph(*Narr1)
#arrLCC = buildArr_lcc_single()

configNP = {'a0':1e-4,'coordSys':'lcc','mass':1.0,'xi':0.0,'dPhi':tempDPHI,'psBounds':[-tempDPHI,tempDPHI],'envelope':'cos','pulseOpt':['analytic'],'deg':2500,'wf':True,'system':'lab'}
configSQ = {'a0':1e-4,'coordSys':'lightcone','mass':1.0,'xi':0.0,'dPhi':tempDPHI,'psBounds':[-tempDPHI,tempDPHI],'envelope':'cos','pulseOpt':['analytic'],'deg':2500,'wf':True,'system':'lab'}

ampObj = amplitude(**configNP)

start = time.time()
resMesh = ampObj(*arrLCC)
#resMesh = ampObj(*arrSPH)
timesNP['calc']=time.time() - start

In [ ]:
print(timesNP['calc'])
print(timesNP['calc']/ampObj.kin.grid.size)

In [ ]:
import itertools
from pathos.multiprocessing import ProcessingPool as Pool


ampObjSQ = sf.diffCSFull(configSQ)


start = time.time()
kinPara = np.array([t for t in itertools.product(arrLCC[0],arrLCC[2],arrLCC[3],arrLCC[1],arrLCC[5],arrLCC[6],arrLCC[4])])
#kinPara = np.array([t for t in itertools.product(arrSPH[0],arrSPH[1],arrSPH[2],arrSPH[3],arrSPH[4],arrSPH[5],arrSPH[6])])
timesSQ['buildKin'] = time.time() - start
pool = Pool(8)
start = time.time()
resSQ = pool.map(ampObjSQ,kinPara)
#resSQ = map(ampObjSQ,kinPara)
timesSQ['calc']=time.time() - start
pool.close()

In [ ]:
print(timesSQ['calc'])
print(timesSQ['calc']/ampObj.kin.grid.size*8)

In [ ]:
print(ampObj.kin.grid.size)

In [ ]:
cShape = resMesh.shape
ind = 0
err = 0
for i0 in np.arange(cShape[0]):
    for i2 in np.arange(cShape[2]):
        for i3 in np.arange(cShape[3]):
            for i1 in np.arange(cShape[1]):
                for i5 in np.arange(cShape[5]):
                    for i6 in np.arange(cShape[6]):
                        for i4 in np.arange(cShape[4]):
                            
                            if not np.isclose(resMesh[i0,i1,i2,i3,i4,i5,i6],resSQ[ind],atol=0.0,rtol=1e-6):
                                err+=1
                                print('-'*20)
                                
                                print('NP',resMesh[i0,i1,i2,i3,i4,i5,i6])
                                print('SQ',resSQ[ind])
                                
                                print()
                                print('\tss ')
                                print('\t\t',arrLCC[0][i0])
                                print('\t\t',kinPara[ind][0])
                            
                                print('\tp2x ')
                                print('\t\t',arrLCC[2][i2])
                                print('\t\t',kinPara[ind][1])
                            
                                print('\tp2y ')
                                print('\t\t',arrLCC[3][i3])
                                print('\t\t',kinPara[ind][2])
                            
                                print('\tp2m ')
                                print('\t\t',arrLCC[1][i1])
                                print('\t\t',kinPara[ind][3])
                            
                                print('\tp3x ')
                                print('\t\t',arrLCC[5][i5])
                                print('\t\t',kinPara[ind][4])
                            
                                print('\tp3y ')
                                print('\t\t',arrLCC[6][i6])
                                print('\t\t',kinPara[ind][5])
                            
                                print('\tp3m ')
                                print('\t\t',arrLCC[4][i4])
                                print('\t\t',kinPara[ind][6])
                            
                            ind+=1
print("Done with %d errors!"%err)